In [33]:
import pandas as pd

base_dir = '../data/mitsui-commodity-prediction-challenge'
train_dir = '/train.csv'
train_labels_dir = '/train_labels.csv'
target_pairs_dir = '/target_pairs.csv'

In [34]:
train_df = pd.read_csv(base_dir + train_dir)
train_df

,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
0,0,2264.5,7205.0,2570.0,3349.0,NaN,NaN,NaN,NaN,NaN,...,1.699987,0.776874,0.888115,0.689954,0.066653,0.090582,0.119630,0.078135,13.822740,0.059163
1,1,2228.0,7147.0,2579.0,3327.0,NaN,NaN,NaN,NaN,NaN,...,1.695279,0.778682,0.889488,0.692628,0.067354,0.091297,0.120520,0.079066,13.888146,0.059895
2,2,2250.0,7188.5,2587.0,3362.0,4684.0,4691.0,4684.0,3363.0,3367.0,...,1.692724,0.780186,0.894004,0.697490,0.067394,0.091478,0.120809,0.079287,13.983675,0.060037
3,3,2202.5,7121.0,2540.0,3354.0,4728.0,4737.0,4729.0,3430.0,3426.0,...,1.683111,0.785329,0.889439,0.698502,0.067639,0.091558,0.121021,0.079285,14.035571,0.059983
4,4,2175.0,7125.0,2604.0,3386.0,NaN,NaN,NaN,NaN,NaN,...,1.684816,0.787264,0.891042,0.701485,0.067443,0.091266,0.121055,0.078925,14.013760,0.059503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,...,1.864661,0.598318,0.827529,0.495125,0.049224,0.072574,0.080968,0.046175,14.058107,0.041388
1913,1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,...,1.863539,0.594400,0.824390,0.490018,0.049409,0.072828,0.080671,0.046113,14.082236,0.041630
1914,1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,...,1.860067,0.595250,0.822392,0.489529,0.049095,0.073232,0.081083,0.045901,14.126606,0.041457
1915,1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,...,1.859624,0.597780,0.817224,0.488520,0.049205,0.073018,0.081170,0.045987,14.095322,0.041368


In [ ]:
import pandas as pd
from itertools import combinations

def create_features(df):
    df_features = df.copy()
    
    lag_cols = [col for col in df.columns if col != 'date_id']
    lag_days = [1, 2, 3, 5, 10, 20]
    windows = [3, 5, 10, 15, 20]
    stats = ['mean', 'std', 'max', 'min']
    change_days = [1, 3, 5, 7, 14, 21]
    
    for col in lag_cols:
        for day in lag_days:
            df_features[f'{col}_lag_{day}'] = df_features[col].shift(day)
            
    for col in lag_cols:
        for window in windows:
            rolling = df_features[col].rolling(window=window).agg(stats)
            rolling.columns = [f'{col}_rolling_{stat}_{window}' for stat in stats]
            df_features = pd.concat([df_features, rolling], axis = 1)
            
    for col in lag_cols:
        for day in change_days:
            df_features[f'change_{day}'] = df_features[col].pct_change(periods=day)
    
    df_features = df_features.ffil()

    return df_features

/var/folders/qz/2h6spgg503g9grggbdgk3fr80000gn/T/ipykernel_75735/3003946325.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df_new[f'{col}_lag_{day}'] = train_df_new[col].shift(day)
/var/folders/qz/2h6spgg503g9grggbdgk3fr80000gn/T/ipykernel_75735/3003946325.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df_new[f'{col}_lag_{day}'] = train_df_new[col].shift(day)
/var/folders/qz/2h6spgg503g9grggbdgk3fr80000gn/T/ipykernel_75735/3003946325.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually

In [45]:
train_df_new

,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_NOKJPY_lag_3,FX_NOKJPY_lag_5,FX_NOKJPY_lag_10,FX_NOKJPY_lag_20,FX_ZARGBP_lag_1,FX_ZARGBP_lag_2,FX_ZARGBP_lag_3,FX_ZARGBP_lag_5,FX_ZARGBP_lag_10,FX_ZARGBP_lag_20
0,0,2264.5,7205.0,2570.0,3349.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2228.0,7147.0,2579.0,3327.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.059163,NaN,NaN,NaN,NaN,NaN
2,2,2250.0,7188.5,2587.0,3362.0,4684.0,4691.0,4684.0,3363.0,3367.0,...,NaN,NaN,NaN,NaN,0.059895,0.059163,NaN,NaN,NaN,NaN
3,3,2202.5,7121.0,2540.0,3354.0,4728.0,4737.0,4729.0,3430.0,3426.0,...,13.822740,NaN,NaN,NaN,0.060037,0.059895,0.059163,NaN,NaN,NaN
4,4,2175.0,7125.0,2604.0,3386.0,4728.0,4737.0,4729.0,3430.0,3426.0,...,13.888146,NaN,NaN,NaN,0.059983,0.060037,0.059895,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,...,14.121359,14.192464,13.920913,13.596089,0.041721,0.041724,0.041370,0.041572,0.040971,0.040051
1913,1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,...,13.977300,14.231042,13.930207,13.590308,0.041388,0.041721,0.041724,0.041060,0.041126,0.039932
1914,1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,...,14.024190,14.121359,13.885341,13.672875,0.041630,0.041388,0.041721,0.041370,0.041188,0.040460
1915,1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,...,14.058107,13.977300,13.991613,13.609097,0.041457,0.041630,0.041388,0.041724,0.041466,0.040372


In [37]:
train_labels = pd.read_csv(base_dir + train_labels_dir)
train_labels

,date_id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,...,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
0,0,0.005948,-0.002851,-0.004675,-0.000639,NaN,NaN,-0.006729,0.006066,NaN,...,NaN,0.021239,-0.005595,NaN,-0.004628,0.033793,NaN,0.038234,NaN,0.027310
1,1,0.005783,-0.024118,-0.007052,-0.018955,-0.031852,-0.019452,0.003002,-0.006876,-0.002042,...,0.003377,0.021372,-0.001517,0.012846,0.010547,0.030527,-0.000764,0.025021,0.003548,0.020940
2,2,0.001048,0.023836,-0.008934,-0.022060,NaN,NaN,0.037449,0.007658,NaN,...,-0.006712,0.009308,0.001857,-0.012761,-0.002345,0.017529,-0.005394,0.004835,-0.009075,0.001706
3,3,0.001700,-0.024618,0.011943,0.004778,NaN,NaN,-0.012519,-0.016896,NaN,...,NaN,0.036880,-0.015189,NaN,0.008118,0.001079,NaN,-0.015102,NaN,-0.033010
4,4,-0.003272,0.005234,0.006856,0.013312,0.023953,0.010681,-0.011649,0.002019,0.003897,...,NaN,0.004937,NaN,-0.006673,-0.016105,-0.004885,NaN,NaN,0.009514,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,-0.012596,0.009309,0.007947,0.006477,-0.026216,-0.029487,-0.003287,0.011751,-0.027610,...,NaN,NaN,NaN,0.042052,NaN,NaN,NaN,NaN,0.031006,NaN
1913,1913,-0.000736,-0.001292,-0.005137,-0.007961,-0.008494,-0.011547,0.004263,0.005670,-0.004822,...,-0.012992,-0.003349,0.010912,0.006198,-0.009216,-0.003038,-0.026082,-0.008057,-0.002069,-0.141053
1914,1914,-0.002294,0.012898,0.009978,0.001567,0.002596,-0.007373,0.007554,0.002661,0.004083,...,-0.005240,0.004044,0.004459,0.002619,0.001308,-0.006772,-0.019918,-0.013304,-0.005527,-0.127688
1915,1915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.008250,0.012842,0.009076,0.000932,0.011613,0.003825,0.024350,-0.006928,0.006805,-0.012187


In [38]:
target_pairs = pd.read_csv(base_dir + target_pairs_dir)
target_pairs

,target,lag,pair
0,target_0,1,US_Stock_VT_adj_close
1,target_1,1,LME_PB_Close - US_Stock_VT_adj_close
2,target_2,1,LME_CA_Close - LME_ZS_Close
3,target_3,1,LME_AH_Close - LME_ZS_Close
4,target_4,1,LME_AH_Close - JPX_Gold_Standard_Futures_Close
...,...,...,...
419,target_419,4,FX_NOKUSD - LME_AH_Close
420,target_420,4,JPX_Gold_Standard_Futures_Close - US_Stock_RY_...
421,target_421,4,US_Stock_EWT_adj_close - LME_AH_Close
422,target_422,4,JPX_Platinum_Standard_Futures_Close - FX_NOKCHF
